In [128]:
import pandas as pd
import json

# reading the first json from the realtime data folder
parent_folder = 'washington_GTFS_data 2024.1.29-2024.2.11/washington_GTFS_data - Copy/output/'
trip_updates = 'RAIL_RT_TRIP_UPDATES/'

file_path = parent_folder + trip_updates + '2024_01_02_00_00_00.json'
try:
	with open(file_path, 'r') as file:
		tripJson = json.load(file)
except FileNotFoundError:
	print(f"File not found: {file_path}")

# extracting the links data from static data
links = pd.read_csv('static/links.csv')
stopTimes = pd.read_csv('static/stop_times.csv')
uniqueLinks = pd.read_csv('static/unique_links.csv')


In [129]:
tripJson

{'header': {'gtfs_realtime_version': '2.0',
  'incrementality': 0,
  'timestamp': 1706745594},
 'entity': [{'id': '0',
   'trip_update': {'trip': {'trip_id': '4982232_19743',
     'start_time': '18:54:00',
     'start_date': '20240131',
     'schedule_relationship': 0,
     'route_id': 'RED',
     'direction_id': 0},
    'stop_time_update': [{'stop_sequence': 1,
      'departure': {'time': 1706745600, 'uncertainty': 0},
      'stop_id': 'PF_A15_C',
      'schedule_relationship': 0},
     {'stop_sequence': 2,
      'arrival': {'time': 1706745792, 'uncertainty': 0},
      'stop_id': 'PF_A14_C',
      'schedule_relationship': 0},
     {'stop_sequence': 3,
      'arrival': {'time': 1706745986, 'uncertainty': 0},
      'stop_id': 'PF_A13_C',
      'schedule_relationship': 0},
     {'stop_sequence': 4,
      'arrival': {'time': 1706746132, 'uncertainty': 0},
      'stop_id': 'PF_A12_C',
      'schedule_relationship': 0},
     {'stop_sequence': 5,
      'arrival': {'time': 1706746289, 'uncert

In [130]:
data = []

for entity in tripJson['entity']:
    if 'stop_time_update' in entity['trip_update']:
        for stop_time_update in entity['trip_update']['stop_time_update']:
            data.append({
                'trip_id': entity['trip_update']['trip']['trip_id'],
                'stop_id': stop_time_update['stop_id'],
                'stop_sequence': stop_time_update['stop_sequence'],
                'arrival_time': stop_time_update.get('arrival', {}).get('time', None),
                'arrival_uncertainty': stop_time_update.get('arrival', {}).get('uncertainty', None),
                'departure_time': stop_time_update.get('departure', {}).get('time', None),
                'departure_uncertainty': stop_time_update.get('departure', {}).get('uncertainty', None),
            })

currentStopTimes = pd.DataFrame(data)

In [131]:
currentStopTimes['arrival_time'] = pd.to_datetime(df['arrival_time'], unit='s', errors='coerce')
currentStopTimes['departure_time'] = pd.to_datetime(df['departure_time'], unit='s', errors='coerce')
currentStopTimes

,trip_id,stop_id,stop_sequence,arrival_time,arrival_uncertainty,departure_time,departure_uncertainty
0,4982232_19743,PF_A15_C,1,NaT,NaN,2024-03-01 11:06:00,0.0
1,4982232_19743,PF_A14_C,2,2024-03-01 11:09:10,0.0,NaT,NaN
2,4982232_19743,PF_A13_C,3,2024-03-01 11:12:24,0.0,NaT,NaN
3,4982232_19743,PF_A12_C,4,2024-03-01 11:14:50,0.0,NaT,NaN
4,4982232_19743,PF_A11_C,5,2024-03-01 11:17:27,0.0,NaT,NaN
...,...,...,...,...,...,...,...
3467,4982449_19743,PF_B07_C,23,2024-03-01 10:44:39,0.0,NaT,NaN
3468,4982449_19743,PF_B08_C,24,2024-03-01 10:46:20,0.0,NaT,NaN
3469,4982449_19743,PF_B09_C,25,2024-03-01 10:47:39,0.0,NaT,NaN
3470,4982449_19743,PF_B10_C,26,2024-03-01 10:49:37,0.0,NaT,NaN


In [ ]:
uniqueLinks

,start_stop,end_stop,mean_speed [km/h],covariance_speed [%],buffer_speed [km/h],mean_time [mins],covariance_time [%],buffer_time [mins],length [km],mean_headway [mins],covariance_headway [%],buffer_headway [mins]
0,PF_A15_C,PF_A14_C,40.065000,1.774532e-16,0.0,4.0,0.0,0.0,2.671000,2.171030,0.594617,3.00
1,PF_A14_C,PF_A13_C,42.096000,1.688916e-16,0.0,3.0,0.0,0.0,2.104800,2.684783,0.605075,4.00
2,PF_A13_C,PF_A12_C,32.964000,0.000000e+00,0.0,2.0,0.0,0.0,1.098800,2.654428,0.656903,4.00
3,PF_A12_C,PF_A11_C,29.072000,0.000000e+00,0.0,3.0,0.0,0.0,1.453600,2.328897,0.579458,2.00
4,PF_A11_C,PF_A10_C,32.418000,2.193121e-16,0.0,4.0,0.0,0.0,2.161200,2.317490,0.577380,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...
197,PF_N11_C,PF_N10_C,34.479600,0.000000e+00,0.0,5.0,0.0,0.0,2.873300,2.990544,0.902491,7.00
198,PF_N10_C,PF_N09_C,31.198628,4.425913e-05,0.0,4.0,0.0,0.0,2.079909,2.778022,0.795745,5.00
199,PF_N09_C,PF_N08_C,34.447983,5.344577e-06,0.0,3.0,0.0,0.0,1.722399,2.941452,0.909301,6.00
200,PF_N08_C,PF_N07_C,26.954000,5.260984e-16,0.0,3.0,0.0,0.0,1.347700,2.906977,0.817076,5.55


In [133]:
# calculating the link travel time, and dwell time

linkPlaceholder = []
for i in range(len(currentStopTimes) - 1):
    if currentStopTimes.loc[i, 'trip_id'] == currentStopTimes.loc[i + 1, 'trip_id']:
        start_stop = currentStopTimes.loc[i, 'stop_id']
        end_stop = currentStopTimes.loc[i + 1, 'stop_id']
        start_sequence = currentStopTimes.loc[i, 'stop_sequence']
        end_sequence = currentStopTimes.loc[i + 1, 'stop_sequence']
        
        link_data = uniqueLinks[(uniqueLinks['start_stop'] == start_stop) & (uniqueLinks['end_stop'] == end_stop)]
        if not link_data.empty:
            length = link_data['length [km]'].values[0]

            # Calculate time difference in minutes
            start_time = currentStopTimes.loc[i, 'departure_time'] if pd.notnull(currentStopTimes.loc[i, 'departure_time']) else currentStopTimes.loc[i, 'arrival_time']
            end_time = currentStopTimes.loc[i + 1, 'arrival_time']
            real_time_taken = (end_time - start_time).total_seconds() / 60

            linkPlaceholder.append({
                'trip_id': currentStopTimes.loc[i, 'trip_id'],
                'start_stop': start_stop,
                'end_stop': end_stop,
                'start_sequence': start_sequence,
                'end_sequence': end_sequence,
                'real_time_taken [mins]': real_time_taken,
                'length': length,
                'start_time': start_time,
                'end_time': end_time
            })

currentLinks = pd.DataFrame(linkPlaceholder)
currentLinks

,trip_id,start_stop,end_stop,start_sequence,end_sequence,real_time_taken [mins],length,start_time,end_time
0,4982232_19743,PF_A15_C,PF_A14_C,1,2,3.166667,2.671000,2024-03-01 11:06:00,2024-03-01 11:09:10
1,4982232_19743,PF_A14_C,PF_A13_C,2,3,3.233333,2.104800,2024-03-01 11:09:10,2024-03-01 11:12:24
2,4982232_19743,PF_A13_C,PF_A12_C,3,4,2.433333,1.098800,2024-03-01 11:12:24,2024-03-01 11:14:50
3,4982232_19743,PF_A12_C,PF_A11_C,4,5,2.616667,1.453600,2024-03-01 11:14:50,2024-03-01 11:17:27
4,4982232_19743,PF_A11_C,PF_A10_C,5,6,3.383333,2.161200,2024-03-01 11:17:27,2024-03-01 11:20:50
...,...,...,...,...,...,...,...,...,...
3319,4982449_19743,PF_B06_C,PF_B07_C,22,23,1.383333,1.827600,2024-03-01 10:43:16,2024-03-01 10:44:39
3320,4982449_19743,PF_B07_C,PF_B08_C,23,24,1.683333,1.454600,2024-03-01 10:44:39,2024-03-01 10:46:20
3321,4982449_19743,PF_B08_C,PF_B09_C,24,25,1.316667,1.778400,2024-03-01 10:46:20,2024-03-01 10:47:39
3322,4982449_19743,PF_B09_C,PF_B10_C,25,26,1.966667,1.679500,2024-03-01 10:47:39,2024-03-01 10:49:37
